# Домашнее задание 2 (5 баллов).

Все задания ниже имеют равный вес (5/16).

In [158]:
import pandas as pd
import numpy as np
from zipfile import ZipFile
import warnings

In [159]:
 warnings.simplefilter(action='ignore', category=Warning)

#### Описание данных

В папке Dat (https://github.com/hse-ds/iad-intro-ds/blob/master/2023/homeworks/Data.zip) находится информация о студентах. Всего 10 групп студентов. Файлы делятся на две категории:
    * Students_info_i - информация о студентах из группы i
    * Students_marks_i - оценки студентов из группы i за экзамены

## Задание 1. 
Соберите всю информацию о студентах в одну таблицу df. В получившейся таблице должна быть информация и оценки всех студентов из всех групп. Напечатайте несколько строк таблицы для демонстрации результата.¶

In [125]:
path = 'Data.zip'

info = {}
marks = {}

with ZipFile(path) as z:
    for file in z.namelist():
        if file.startswith('Data/Students_info'):
            with z.open(file) as f:
                info[file] = pd.read_csv(f)
        elif file.startswith('Data/Students_marks'):
            with z.open(file) as f:
                marks[file] = pd.read_csv(f)

In [126]:
df1 = pd.concat(info.values(), ignore_index=True)
df2 = pd.concat(marks.values(), ignore_index=True)
df = pd.concat([df1, df2], axis=1)
df.head(3)

,index,gender,race/ethnicity,parental level of education,lunch,test preparation course,group,index,math score,reading score,writing score
0,0,female,group B,bachelor's degree,standard,none,group1,0,72,72,74
1,1,female,group C,some college,standard,completed,group1,1,69,90,88
2,2,female,group B,master's degree,standard,none,group1,2,90,95,93


## Задание 2. 
Удалите столбец index у полученной таблицы. Напечатайте первые 10 строк таблицы.

In [127]:
df.drop(columns=['index'], inplace=True)
df.head(3)

,gender,race/ethnicity,parental level of education,lunch,test preparation course,group,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,group1,72,72,74
1,female,group C,some college,standard,completed,group1,69,90,88
2,female,group B,master's degree,standard,none,group1,90,95,93


## Задание 3. 
Выведите на экран размеры полученной таблицы

In [128]:
df.shape

(1000, 9)

## Задание 4. 
Выведите на экран статистические характеристики числовых столбцов таблицы (минимум, максимум, среднее значение, стандартное отклонение)

In [129]:
df.select_dtypes('number').agg(['mean', 'max', 'min', 'std'])

,math score,reading score,writing score
mean,66.08900,69.169000,68.054000
max,100.00000,100.000000,100.000000
min,0.00000,17.000000,10.000000
std,15.16308,14.600192,15.195657


## Задание 5. 
Проверьте, есть ли в таблице пропущенные значения

In [130]:
df.isna().sum()

gender                         0
race/ethnicity                 0
parental level of education    0
lunch                          0
test preparation course        0
group                          0
math score                     0
reading score                  0
writing score                  0
dtype: int64

## Задание 6. 
Выведите на экран средние баллы студентов по каждому предмету (math, reading, writing)

In [131]:
df[['math score', 'reading score', 'writing score']].mean()

math score       66.089
reading score    69.169
writing score    68.054
dtype: float64

## Задание 7. 
Как зависят оценки от того, проходил ли студент курс для подготовки к сдаче экзамена (test preparation course)? Выведите на экран для каждого предмета в отдельности средний балл студентов, проходивших курс для подготовки к экзамену и не проходивших курс.**

In [132]:
df.groupby('test preparation course')[['math score', 'reading score', 'writing score']].mean()

,math score,reading score,writing score
test preparation course,,,
completed,69.695531,73.893855,74.418994
none,64.077882,66.534268,64.504673


Как видим, студенты, прошедшие подготовительный курс, сдали экзамен силно лучше

## Задание 8. 
Выведите на экран все различные значения из столбца lunch.

In [133]:
print(*list(df['lunch'].unique()), sep='\n')

standard
free/reduced


## Задание 9.
Переименуйте колонку "parental level of education" в "education", а "test preparation course" в "test preparation" с помощью метода pandas rename

In [134]:
df = df.rename(columns={'parental level of education': 'education', 'test preparation course': 'test preparation'})
df.head(3)

,gender,race/ethnicity,education,lunch,test preparation,group,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,group1,72,72,74
1,female,group C,some college,standard,completed,group1,69,90,88
2,female,group B,master's degree,standard,none,group1,90,95,93


**Зафиксируем минимальный балл для сдачи экзамена**

In [135]:
passmark = 50

## Задание 10.
    * Какая доля студентов сдала экзамен по математике (passmark >= 50)?
    * Какая доля студентов, проходивших курс подготовки к экзамену, сдала экзамен по математике?
    * Какая доля женщин, не проходивших курс подготовки к экзамену, не сдала экзамен по математике? 

In [136]:
ans = df[df['math score'] >= 50].shape[0] / df.shape[0]
print(f'Доля сдавших математику: {ans}')

Доля сдавших математику: 0.865


In [137]:
prep_done = df[df['test preparation'] == 'completed'].shape[0]
math_done = df[(df['test preparation'] == 'completed') & (df['math score'] >= 50)].shape[0]
ans = math_done / prep_done
print(f'Доля сдавших математику среди тех, кто прошел курс подготовки к экзамену: {ans}')

Доля сдавших математику среди тех, кто прошел курс подготовки к экзамену: 0.9217877094972067


In [138]:
women = df[(df['gender'] == 'female') & (df['test preparation'] == 'none')].shape[0]
women_failed = df[(df['gender'] == 'female') & (df['test preparation'] == 'none') & (df['math score'] < 50)].shape[0]
ans = women_failed / women
print(f'Доля не сдавших математику женщин среди тех, кто не прошел курс подготовки: {ans}')

Доля не сдавших математику женщин среди тех, кто не прошел курс подготовки: 0.20958083832335328


## Задание 11. 
**С помощью groupby выполните задания ниже. Также выведите время выполнения каждого из заданий.**

    * Для каждой этнической группы выведите средний балл за экзамен по чтению
    * Для каждого уровня образования выведите минимальный балл за экзамен по письму

In [139]:
%%time
df.groupby('race/ethnicity')['reading score'].mean()

CPU times: total: 0 ns
Wall time: 981 μs


race/ethnicity
group A    64.674157
group B    67.352632
group C    69.103448
group D    70.030534
group E    73.028571
Name: reading score, dtype: float64

In [140]:
%%time
df.groupby('education')['writing score'].min()

CPU times: total: 0 ns
Wall time: 1 ms


education
associate's degree    35
bachelor's degree     38
high school           15
master's degree       46
some college          19
some high school      10
Name: writing score, dtype: int64

## Задание 12. 
Выполните задание 11 с помощью циклов. Сравните время выполнения.

* Для каждой этнической группы выведите средний балл за экзамен по чтению

In [141]:
%%time

groups = {}

for i, row in df.iterrows():
    group = row['race/ethnicity']
    score = row['reading score']
    if not group in groups:
        groups[group] = [score, 1]
    else:
        groups[group][0] += score
        groups[group][1] += 1
        
for k, v in groups.items():
    print(f'{k}: {v[0] / v[1]}')

group B: 67.35263157894737
group C: 69.10344827586206
group A: 64.67415730337079
group D: 70.03053435114504
group E: 73.02857142857142
CPU times: total: 46.9 ms
Wall time: 45 ms


* Для каждого уровня образования выведите минимальный балл за экзамен по письму

In [142]:
%%time

min_score = {}

for i, row in df.iterrows():
    ed = row['education']
    score = row['writing score']
    if not ed in min_score:
        min_score[ed] = score
    else:
        if score < min_score[ed]:
            min_score[ed] = score
        
for k, v in min_score.items():
    print(f'{k}: {v}')

bachelor's degree: 38
some college: 19
master's degree: 46
associate's degree: 35
high school: 15
some high school: 10
CPU times: total: 46.9 ms
Wall time: 44 ms


Питон сильно медленее методов пандаса!

## Задание 13. 
Выведите на экран средние баллы студентов по каждому предмету в зависимости от пола и уровня образования. То есть должно получиться количество групп, равных 2 * (число уровней образования), и для каждой такой группы выыведите средний балл по каждому из предметов.

In [143]:
df.head(1)

,gender,race/ethnicity,education,lunch,test preparation,group,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,group1,72,72,74


In [149]:
df.pivot_table(index=['gender', 'education'],
               values=['math score', 'reading score', 'writing score'],
               aggfunc='mean')

math score  reading score  writing score
gender education                                                   
female associate's degree   65.250000      74.120690      74.000000
       bachelor's degree    68.349206      77.285714      78.380952
       high school          59.351064      68.202128      66.691489
       master's degree      66.500000      76.805556      77.638889
       some college         65.406780      73.550847      74.050847
       some high school     59.296703      69.109890      68.285714
male   associate's degree   70.764151      67.433962      65.405660
       bachelor's degree    70.581818      68.090909      67.654545
       high school          64.705882      61.480392      58.539216
       master's degree      74.826087      73.130435      72.608696
       some college         69.009259      64.990741      63.148148
       some high school     67.840909      64.693182      61.375000

## Задание 14. 
Сколько студентов успешно сдали экзамен по математике?

Создайте новый столбец в таблице df под названием Math_PassStatus и запишите в него F, если студент не сдал экзамен по математике (балл за экзамен < passmark), и P иначе.

Посчитайте количество студентов, сдавших и не сдавших экзамен по математике.

Сделайте аналогичные шаги для экзаменов по чтению и письму.

In [152]:
df['Math_PassStatus'] = df['math score'].apply(lambda x: 'F' if x < passmark else 'P')
df['Reading_PassStatus'] = df['reading score'].apply(lambda x: 'F' if x < passmark else 'P')
df['Writing_PassStatus'] = df['writing score'].apply(lambda x: 'F' if x < passmark else 'P')
df.head(1)

,gender,race/ethnicity,education,lunch,test preparation,group,math score,reading score,writing score,Math_PassStatus,Reading_PassStatus,Writing_PassStatus
0,female,group B,bachelor's degree,standard,none,group1,72,72,74,P,P,P


In [164]:
math_p = df['Math_PassStatus'].value_counts()[0]
math_f = df['Math_PassStatus'].value_counts()[1]
read_p = df['Reading_PassStatus'].value_counts()[0]
read_f = df['Reading_PassStatus'].value_counts()[1]
write_p = df['Writing_PassStatus'].value_counts()[0]
write_f = df['Writing_PassStatus'].value_counts()[1]

print(f'Математика\nСдало: {math_p}\nНе сдало: {math_f}\n')
print(f'Чтение\nСдало: {read_p}\nНе сдало: {read_f}\n')
print(f'Письмо\nСдало: {write_p}\nНе сдало: {write_f}')

Математика
Сдало: 865
Не сдало: 135

Чтение
Сдало: 910
Не сдало: 90

Письмо
Сдало: 886
Не сдало: 114


## Задание 15. Сколько студентов успешно сдали все экзамены?

Создайте столбец OverAll_PassStatus и запишите в него для каждого студента 'F', если студент не сдал хотя бы один из трех экзаменов, а иначе 'P'.

Посчитайте количество студентов, которые сдали все экзамены.

In [167]:
df

,gender,race/ethnicity,education,lunch,test preparation,group,math score,reading score,writing score,Math_PassStatus,Reading_PassStatus,Writing_PassStatus
0,female,group B,bachelor's degree,standard,none,group1,72,72,74,P,P,P
1,female,group C,some college,standard,completed,group1,69,90,88,P,P,P
2,female,group B,master's degree,standard,none,group1,90,95,93,P,P,P
3,male,group A,associate's degree,free/reduced,none,group1,47,57,44,F,P,F
4,male,group C,some college,standard,none,group1,76,78,75,P,P,P
...,...,...,...,...,...,...,...,...,...,...,...,...
995,female,group E,master's degree,standard,completed,group10,88,99,95,P,P,P
996,male,group C,high school,free/reduced,none,group10,62,55,55,P,P,P
997,female,group C,high school,free/reduced,completed,group10,59,71,65,P,P,P
998,female,group D,some college,standard,completed,group10,68,78,77,P,P,P


In [176]:
df['OverAll_PassStatus'] = df.apply(lambda row: 'F' if (row['Math_PassStatus'] == 'F' 
                                    or row['Reading_PassStatus'] == 'F' 
                                    or row['Writing_PassStatus'] == 'F') else 'P', axis=1)

ans = df[df['OverAll_PassStatus'] == 'P'].shape[0]
print(f'Кол-во студентов, сдавших все экзамены: {ans}')

Кол-во студентов, сдавших все экзамены: 812


## Задание 16. 
Переведем баллы в оценки

####    больше 90 = A
####      80-90 = B
####      70-80 = C
####      60-70 = D
####      50-60 = E
####    меньше 50 = F (Fail)

Создайте вспомогательную функцию, которая будет по среднему баллу за три экзамена выставлять оценку студенту по данным выше критериям.

Создайте столбец Grade и запишите в него оценку каждого студента.

Выведите количество студентов, получивших каждую из оценок.

**В случае, если средний балл попадает на границу между оценками (т.е. равен ровно 60, 70 или 80 баллов), вы можете интерпретировать условие на своё усмотрение (т.е. можете поставить за 60 баллов оценку D, а можете - E).**

In [177]:
def get_grade(grade):
    if grade >= 90:
        return 'A'
    elif grade >= 80:
        return 'B'
    elif grade >= 70:
        return 'C'
    elif grade >= 60:
        return 'D'
    elif grade >= 50:
        return 'E'
    else:
        return 'F'

In [181]:
df['Grade'] = df.apply(lambda row: get_grade((row['math score'] + row['reading score'] + row['writing score']) / 3), axis=1)

In [182]:
df.head(3)

,gender,race/ethnicity,education,lunch,test preparation,group,math score,reading score,writing score,Math_PassStatus,Reading_PassStatus,Writing_PassStatus,OverAll_PassStatus,Grade
0,female,group B,bachelor's degree,standard,none,group1,72,72,74,P,P,P,P,C
1,female,group C,some college,standard,completed,group1,69,90,88,P,P,P,P,B
2,female,group B,master's degree,standard,none,group1,90,95,93,P,P,P,P,A
